**Goal:** Tokenize a sentence into words and punctuation and keep track of the start/end indices of each token - needed to string together sequences of tokens to join `B-`, `I-` tags.

In [12]:
import os
import sys
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
s = "A 36-year old patient was admitted complaining of stomach pain and was later diagnosed with ulcerative colitis."

In [3]:
def split_str_token(s) : 
    istart=0
    iend=0
    ovec=[]
    for i in range(len(s)) : 
        if str(s[i]).lower() in list('abcdefghijklmnopqrstuvwxyz1234567890') :
            continue
        elif s[i] in list('-/.?,!%*') :
            
            # Capture the term before it
            iend   = i
            ovec.append([s[istart:iend],istart,iend])
            istart = iend+1
            
            # Now capture the punctuation
            istart = i
            iend   = i+1
            ovec.append([s[i],istart,iend])
            istart = i+1
            continue
        else : 
            iend   = i
            ovec.append([s[istart:iend],istart,iend])
            istart = iend+1
    return ovec

In [4]:
print(s)

A 36-year old patient was admitted complaining of stomach pain and was later diagnosed with ulcerative colitis.


In [5]:
# Turn it into an array to keep track of start, end indices
tok_mat = np.array([np.array(isi) for isi in split_str_token(s)])

In [6]:
words = tok_mat[:,0]
sind  = tok_mat[:,1]
eind  = tok_mat[:,2]

In [7]:
if False : 
    fout_words = open("words.txt","w")
    fout_ind   = open("indices.txt","w")
    for i in range(len(words)) :
        #print(words[i]+","+sind[i]+","+eind[i])
        fout_words.write(words[i]+" O \n")
        fout_ind.write(sind[i]+","+eind[i]+"\n")
        if i>0 and i%10 == 0 :
            fout_words.write("\n")
            fout_ind.write("\n")
    fout_words.close()
    fout_ind.close()

## Re-ingest the words and indices and try to tag the sentence with it

In [8]:
tokens  = []
ind     = []

with open("words.txt","r") as fin :
    tokens = fin.readlines()
with open("indices.txt","r") as fin :
    ind   = fin.readlines()
assert(len(tokens)==len(ind))    

In [9]:
words   = []
tags    = []
sind    = []
eind    = []

for i in range(len(tokens)) : 
    if len(tokens[i].split())==0 :
        continue
    words.append(tokens[i].split()[0])
    tags.append(tokens[i].split()[1].replace("\n",""))
    sind.append(int(ind[i].split(",")[0]))
    eind.append(int(ind[i].split(",")[1]))
    
assert(len(words)==len(tags)==len(sind)==len(eind))

In [25]:
iannot=-1
for pii_type in ['AGE','MEDICAL_HISTORY'] : 
    for i in range(len(tags)) :
        if tags[i] == 'B-%s'%(pii_type) : 
            iannot+=1
            istart = sind[i]
            iend   = eind[i]
            # Iterate forward to see if I-MEDICAL_HISTORY is found
            for j in range(i+1,len(tags)) : 
                if tags[j] == 'I-%s'%(pii_type) :
                    iend = eind[j]
                else :
                    break
            print("%s tag found: %s at [%i,%i]"%(pii_type,s[istart:iend],istart,iend))
            e = ET.fromstring('<annotation id="%i" startIndex="%i" \
            endIndex="%i"><semanticType>%s</semanticType></annotation>'%(iannot,int(istart),int(iend),pii_type))
            ET.dump(e)

AGE tag found: 36 at [2,4]
<annotation endIndex="4" id="0" startIndex="2"><semanticType>AGE</semanticType></annotation>
MEDICAL_HISTORY tag found: stomach pain at [50,62]
<annotation endIndex="62" id="1" startIndex="50"><semanticType>MEDICAL_HISTORY</semanticType></annotation>
MEDICAL_HISTORY tag found: ulcerative colitis at [92,110]
<annotation endIndex="110" id="2" startIndex="92"><semanticType>MEDICAL_HISTORY</semanticType></annotation>
